# Import Libraries

**Shapefile:** This is the pyshp library that is used to manipulate shapefiles.
**Math:** This library is used for basic math functions to calculate distance between points

In [1]:
import pandas as pd
import shapefile as sf
import math as math

In [2]:
line_stops = pd.read_csv('../data/processed/assignment1/line_stops.csv')
sf_actu_lines = sf.Reader('../data/raw/shapefiles/ACTU_LINES.shp')
# here we initialize shape_records, which includes a combination of the shapes and records from the shapefile. This combination will allow us to pull the lambert coordinates from the shapes as while also accessing the record information like line_id.
shape_records = sf_actu_lines.shapeRecords()

# Shapefile Distance Calculation Function

Now that we have our libraries loaded and files imported, we will create a function that can calculate the distance between two points on a polyline. The start_point and end_point are indexes to tell us where we should start and stop calculating distance in the polyline. the line_segment is one of the shape elements that will be pulled from the shapefile. This calculation will be called later in an iterative for loop for each shape element withing shape_records.shape.

We will calculate the distance between each point in the shapefile using Pythagoreas' theorem, since the units in both line_stops and the shapefile are already provided in Belgium Lambert 1972 format, which projects the points onto a flat surface.

In [3]:
def calculate_distance_between_polyline_points(start_point: int, stop_point: int, line_segment: sf.Shape) -> float:
    # initializing our total distance to 0
    total_distance = 0
    # we'll need to calculate the distance between each consecutive pair of coordinates, and will iterate
    # from the start_point to the end_point. Each newly caluclated distance between points will be added
    # to the sum total_distance and then returned.
    for index in range(start_point,stop_point-1):
        current = index
        next = index + 1
        total_distance += math.sqrt(pow((line_segment.points[current][0] - line_segment.points[next][0]),2) + pow((line_segment.points[current][1] - line_segment.points[next][1]),2))
    return total_distance


# Calculate Distance Between Stops

Time to get to work! Here we several nested for loops that are used to compare match up the line in the shapefile to the line in the line_stops. For each matching line, we will cycle through to project the stop location ONTO the polyline. This is required because a bus stop can be imagined to be on a sidewalk, while the polyline is moving along the road.

Once we have matched up our bus stop with the nearest polyline point, we move to the next stop and do the same. Having 2 stop locations projected, we can call the previously defined calculate_distance_between_polyline_points function to find the distance between these 2 stops.

The first iteration of the loop will result in a dummy value, as it does not have a real stop to pair with. All of these dummy values are dropped once the dictionary that stores all values is transformed into a dataframe.

The data frame will be accessed latter with the unique combination of [LineID + fromStopID + toStopID]. This combination will be different depending on which direction a vehicle is moving, as the stop id's are not the same on each side of a street.

We will also hold onto the index value for the polyline location in case we need it later on for future predictions.

In [49]:
# initialize a dictionary that will be used to make a dataframe and csv file with the following format:
# |    LineId    |   FromStop  |   ToStop  |   distance    |    fromIndex   |    toIndex   |
stop_distance = {'LineId': [], 'FromStop': [], 'ToStop': [], 'distance': [], 'fromIndex': [], 'toIndex': [], 'FromStop_lat': [], 'FromStop_lon': [], 'ToStop_lat': [], 'ToStop_lon': []}

# Initializing variables that will be used in loops
last_pointID = 0
last_stop_id = 0
last_stop_lat = 0
last_stop_lon = 0
adjusted_stop_lat_GPS = 0
adjusted_stop_lon_GPS = 0

geo_json_test = {"toIndex": [], "geojson": [], "WKT": []}
# look through each shape/record combo in the shape_records file. Each element of shape_records represents a single line (metro, bus or tram)
for shape_record in shape_records:
    record = shape_record.record
    shape = shape_record.shape
    # look through each of the stops that exist in the line_stops csv. Here we are going to only cycle through a subset of the line_stops where there is a match on LineId and the direction to reduce computation time.
    for index, stop in line_stops[(line_stops['lineId'] == record['LIGNE']) & (line_stops['direction'] == record['VARIANTE'])].sort_values(by=['order']).iterrows():
        # Initializing variables that will be used in loops
        min_distance = 50
        adjusted_stop_lat = 50
        adjusted_stop_lon = 50
        current_pointID = 0
        current_stop_id = stop['stop_id_int']
        stop_lat = stop['lambert_x']
        stop_lon = stop['lambert_y']
        #After choosing a single stop from the line_stops file, we will compare that stops lambert GPS position to each coordinate that makes up the polyline in the current shape_records shape. We are finding the closest location in the shape file to our bus stop location. This can be done using euclidean distance calculation because the coordinates are in lambert notation. Whichever location on the polyline is the closest becomes the projected location of the bus stop using the if statement.
        for pointID in range(len(shape.points)):
            point_lat = shape.points[pointID][0]
            point_lon = shape.points[pointID][1]
            distance = math.sqrt(pow((point_lat - stop_lat),2) + pow((point_lon - stop_lon),2))
            # if statement to compare distances and updated if shorter. It also saves the polyline info for future use in predicting
            # which method of transport is being used.
            if distance < min_distance:
                min_distance = distance
                adjusted_stop_lat = point_lat
                adjusted_stop_lon = point_lon
                current_pointID = pointID
                adjusted_stop_lat_GPS = stop['lat']
                adjusted_stop_lon_GPS = stop['long']
        # now we call a previously defined function to calculate the total distance between the location projected during the previous for loop iteration and the current loop iteration. We are able to do this because the stops have been sorted by descending order from first to last. The first row in the array will always be a dummy row and needs to be dropped afterwards.
        distance_between_stops = calculate_distance_between_polyline_points(last_pointID, current_pointID, shape)
        # we update our dictionary with all the values needed for distance between stops.
        # we will also strip out the leading zeros and the trailing text characters indicating (b,t,m for bus, tram and metro)
        stop_distance['LineId'].append(stop['lineId'][:-1].strip("0"))
        stop_distance['FromStop'].append(last_stop_id)
        stop_distance['ToStop'].append(current_stop_id)
        stop_distance['distance'].append(distance_between_stops)
        stop_distance['fromIndex'].append(last_pointID)
        stop_distance['toIndex'].append(current_pointID)
        geo_json_test['toIndex'].append(last_pointID)
        stop_distance['FromStop_lat'].append(last_stop_lat)
        stop_distance['FromStop_lon'].append(last_stop_lon)
        stop_distance['ToStop_lat'].append(adjusted_stop_lat_GPS)
        stop_distance['ToStop_lon'].append(adjusted_stop_lon_GPS)
        # after calculating the distance, we update the last stop id, point, and lat/lon to the currently being used before iterating through to the next bus stop. The current point becomes the last point for the next calculation.
        polyline_list = []
        WKT_list = []
        for pointID in range(last_pointID, current_pointID+1):
            point_lat = shape.points[pointID][0]
            point_lon = shape.points[pointID][1]
            polyline_list.append([point_lat, point_lon])
            WKT_list.append(f"{point_lat} {point_lon}")
        last_stop_id = current_stop_id
        last_pointID = current_pointID
        last_stop_lat = adjusted_stop_lat_GPS
        last_stop_lon = adjusted_stop_lon_GPS
        # for loop to add polyline coordinates to speed table for visualization
        # HERE is where I create a string that can be used to create a polyline in the visualization.
        # Using geo_json_test to figure it out, as I was having issues and could not resolve why my
        # points were not showing up. Formatting should be ok, just need lambert to EPSG 4326 conversion
        # pyproj (see below) could be used for this.
        geo_json_test["geojson"].append(f'{{"type": "FeatureCollection", "features": [{{"type": "Feature", properties: {{}}, "geometry": {{"type": "LineString", "coordinates": {polyline_list}}}}}]}}')
        geo_json_test["WKT"].append(f'LINESTRING ({WKT_list})')
# now we convert the dictionary to a Pandas DataFrame for easier manipulation
df_stop_distance = pd.DataFrame.from_dict(stop_distance)
df_stop_distance.drop(df_stop_distance[df_stop_distance['toIndex'] == 0].index, inplace = True)

# Export to CSV

We can now export the dataframe to a csv file for use in other parts of the cleaning and predicitons.

In [36]:
# finally, we export the distance to a csv file named stop_distance.csv
df_stop_distance.to_csv (r'../data/processed/assignment1/stop_distance.csv', index = False, header=True)


In [50]:
df_geo_json_test = pd.DataFrame.from_dict(geo_json_test)
df_geo_json_test.drop(df_geo_json_test[df_geo_json_test['toIndex'] == 0].index, inplace = True)
df_geo_json_test.to_csv (r'../data/processed/assignment1/geo_json_test.csv', index = False, header=True)

## Can use shapely to transform geometry object from one EPSG to another.

import pyproj
from pyproj import Transformer
import shapely.ops as sp_ops

def  transform_geom_to_srid(geom, scrs, tcrs)
    """ Transform a geometry to a new target CRS.
        Works with pyproj version >= 2.x.x
        - geom is a Shapely geometry instance
        - scrs is your input CRS EPSG integer (the one of your original data)
        - tcrs is your target CRS EPSG integer (the one you want to reproject
                  your data in, probably 4326 in your case)
    """
    project = Transformer.from_crs(
        'EPSG:'+str(scrs),
        'EPSG:'+str(tcrs),
         always_xy=True
    )
    return sp_ops.transform(project.transform, geom)